
Circuit Simulation Example with OBI-One
======================================

This notebook demonstrates how to run circuit simulations using the obi-one simulation framework
with [BlueCelluLab](https://github.com/openbraininstitute/BlueCelluLab) as the simulator.

In [1]:
import requests
import obi_one as obi
virtual_lab_id=obi.LAB_ID_STAGING_TEST
project_id=obi.PROJECT_ID_STAGING_TEST
from entitysdk import Client, ProjectContext

from obi_auth import get_token
from obi_notebook import get_projects

obi_one_api_url = "http://127.0.0.1:8100"

token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=obi.LAB_ID_STAGING_TEST, project_id=obi.PROJECT_ID_STAGING_TEST)
db_client = Client(api_url="https://staging.openbraininstitute.org/api/entitycore", project_context=project_context, token_manager=token)

In [2]:
# # Get SONATA circuit object
# c = circuit.sonata_circuit
# print("..." + str(c.node_sets.content)[-25:])

# # Adding a node set to the circuit
# obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})
# print("..." + str(c.node_sets.content)[-55:])

# # Adding a node set with an exising name => NOT POSSIBLE
# # obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": {"layer": [2, 3]}})  # AssertionError: Node set 'Layer23' already exists!

# # Update/overwrite an existing node set
# obi.NeuronSet.add_node_set_to_circuit(c, {"Layer23": ["Layer2", "Layer3"]}, overwrite_if_exists=True)  # Update/overwrite
# print("..." + str(c.node_sets.content)[-58:])

# # Adding multiple node sets
# obi.NeuronSet.add_node_set_to_circuit(c, {"Layer45": ["Layer4", "Layer5"], "Layer56": ["Layer5", "Layer6"]})
# print("..." + str(c.node_sets.content)[-124:])

# # Add node set from NeuronSet object, resolved in circuit's default node population
# neuron_set = obi.CombinedNeuronSet(node_sets=("Layer1", "Layer2", "Layer3"))
# obi.NeuronSet.add_node_set_to_circuit(c, {"Layer123": neuron_set.get_node_set_definition(circuit, circuit.default_population_name)})
# print("..." + str(c.node_sets.content)[-168:])

# # Adding a node sets based on previously added node sets
# obi.NeuronSet.add_node_set_to_circuit(c, {"AllLayers": ["Layer123", "Layer4", "Layer56"]})
# print("..." + str(c.node_sets.content)[-216:])

# # Write new circuit's node set file
# obi.NeuronSet.write_circuit_node_set_file(c, output_path="./", file_name="new_node_sets.json", overwrite_if_exists=True)

Now, let's generate the Simulation Config

In [3]:
from pathlib import Path
import obi_one as obi

# === Parameters ===
output_path = Path(".")
sim_duration = 3000.0

# === 1. Build Form (CircuitSimulationScanConfig) ===
sim_form = obi.CircuitSimulationScanConfig.empty_config()

# Info
info = obi.Info(
    campaign_name="Small Microcircuit Simulation",
    campaign_description="Simulation of circuit with predefined neuron set and constant current stimulus"
)
sim_form.set(info, name="info")

# Neuron Sets
sim_nset = obi.AllNeurons()
sim_form.add(sim_nset, name="All Biophys")

# Timestamps
timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=1, interval=100)
sim_form.add(timestamps, name="Timestamps")

# Stimulus
stimulus = obi.PoissonSpikeStimulus(
    duration=800.0,
    timestamps=timestamps.ref,
    frequency=20,
    source_neuron_set=sim_nset.ref,
    targeted_neuron_set=sim_nset.ref
)
sim_form.add(stimulus, name="PoissonInput")

stimulus = obi.ConstantCurrentClampSomaticStimulus(
    timestamps=timestamps.ref, duration=2000.0, neuron_set=sim_nset.ref, amplitude=0.5
)
sim_form.add(stimulus, name="CurrentClampInput")

# Recording
recording = obi.SomaVoltageRecording(neuron_set=sim_nset.ref)
sim_form.add(recording, name="SomaVoltage")

# Initialization block
init = obi.CircuitSimulationScanConfig.Initialize(
    circuit=obi.CircuitFromID(id_str="13a54362-0c99-43fd-94e4-58bdb5b79466"),
    node_set=sim_nset.ref,
    simulation_length=sim_duration,
)
sim_form.set(init, name="initialize")
# Validated Config
validated_sim_conf = sim_form.validated_config()

print(validated_sim_conf)

# === 2. Wrap into a Simulation ===
grid_scan = obi.GridScanGenerationTask(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../../obi-output/run_circuit_simulations/grid_scan')
grid_scan.execute(db_client=db_client)
obi.run_tasks_for_generated_scan(grid_scan, db_client=db_client)

CircuitSimulationScanConfig(type='CircuitSimulationScanConfig', timestamps={'Timestamps': RegularTimestamps(type='RegularTimestamps', start_time=0.0, interval=100.0, number_of_repetitions=1)}, stimuli={'PoissonInput': PoissonSpikeStimulus(type='PoissonSpikeStimulus', timestamps=TimestampsReference(block_dict_name='timestamps', block_name='Timestamps', type='TimestampsReference'), source_neuron_set=NeuronSetReference(block_dict_name='neuron_sets', block_name='All Biophys', type='NeuronSetReference'), targeted_neuron_set=NeuronSetReference(block_dict_name='neuron_sets', block_name='All Biophys', type='NeuronSetReference'), timestamp_offset=0.0, duration=800.0, frequency=20.0, random_seed=0), 'CurrentClampInput': ConstantCurrentClampSomaticStimulus(type='ConstantCurrentClampSomaticStimulus', timestamps=TimestampsReference(block_dict_name='timestamps', block_name='Timestamps', type='TimestampsReference'), neuron_set=NeuronSetReference(block_dict_name='neuron_sets', block_name='All Biophys'

In [4]:
# population_name = circuit.default_population_name
simulation_config_path = grid_scan.single_configs[0].coordinate_output_root / "simulation_config.json"
print(simulation_config_path)

circuit_folder = grid_scan.single_configs[0].coordinate_output_root / "sonata_circuit"

../../../obi-output/run_circuit_simulations/grid_scan/0/simulation_config.json


Get the path to the mod files and compile the mod files. 

In [5]:
# Remove the old compiled mod files folder
! rm -r arm64/
# flag DISABLE_REPORTINGLIB to skip SonataReportHelper.mod and SonataReport.mod from compilation.
!../../.venv/bin/nrnivmodl -incflags "-DDISABLE_REPORTINGLIB" {circuit_folder}/mod

/usr/bin/xcrun
/Users/james/Documents/obi/code/obi-one/examples/E_run_small_microcircuit
Mod files: "../../../obi-output/run_circuit_simulations/grid_scan/0/sonata_circuit/mod/../../../obi-output/run_circuit_simulations/grid_scan/0/sonata_circuit/mod/CaDynamics_DC0.mod" "../../../obi-output/run_circuit_simulations/grid_scan/0/sonata_circuit/mod/../../../obi-output/run_circuit_simulations/grid_scan/0/sonata_circuit/mod/Ca_HVA2.mod" "../../../obi-output/run_circuit_simulations/grid_scan/0/sonata_circuit/mod/../../../obi-output/run_circuit_simulations/grid_scan/0/sonata_circuit/mod/Ca_LVAst.mod" "../../../obi-output/run_circuit_simulations/grid_scan/0/sonata_circuit/mod/../../../obi-output/run_circuit_simulations/grid_scan/0/sonata_circuit/mod/ConductanceSource.mod" "../../../obi-output/run_circuit_simulations/grid_scan/0/sonata_circuit/mod/../../../obi-output/run_circuit_simulations/grid_scan/0/sonata_circuit/mod/DetAMPANMDA.mod" "../../../obi-output/run_circuit_simulations/grid_scan/0/s

Run circuit simulation using BlueCelluLab backend. In future, we will support Neurodamus backend as well. This will run a SONATA simulation.  

In [6]:
# Run the simulation
from obi_one.scientific.library.simulation_execution import run
run(
    simulation_config=simulation_config_path,
    simulator="bluecellulab", # Optional: bluecellulab or neurodamus. Default: bluecellulab
    save_nwb=False            # Optional: Save results in NWB format. Default: False
)

[2025-10-03 10:28:29,526] INFO: Starting simulation with bluecellulab backend
Try loading libmpi
load_mpi: dlopen(libmpi.dylib, 0x000A): tried: 'libmpi.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OSlibmpi.dylib' (no such file), '/Users/james/Documents/obi/code/obi-one/.venv/lib/python3.12/site-packages/neuron/.data/lib/../lib/libmpi.dylib' (no such file), '/usr/lib/libmpi.dylib' (no such file, not in dyld cache), 'libmpi.dylib' (no such file)
Is openmpi or mpich installed? If not in default location, need a LD_LIBRARY_PATH on Linux or DYLD_LIBRARY_PATH on Mac OS. On Mac OS, full path to a MPI library can be provided via environmental variable MPI_LIB_NRN_PATH
could not dynamically load libmpi.so or libmpich.so

[2025-10-03 10:28:29,527] INFO: File logging initialized. Log file: logs/simulation_20251003_082829.log
[2025-10-03 10:28:29,527] INFO: Initializing BlueCelluLab simulation
[2025-10-03 10:28:29,528] INFO: Rank 0 node IDs: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
[2025-10-03 

--No graphics will be displayed.


[2025-10-03 10:28:31,387] INFO: Added                                                                SynapseProperty.PRE_GID  \
edge_name                                          synapse_id                            
external_S1nonbarrel_neurons__S1nonbarrel_neuro... 0                                39   
                                                   1                                39   
                                                   2                                39   
                                                   3                                71   
                                                   4                                71   
...                                                                                ...   
                                                   2372                           7401   
                                                   2373                           7415   
                                                   2374       

ValidationError: 1 validation error for SynapseReplay
spike_file
  Value error, spike_file PoissonInput_spikes.h5 does not exist [type=value_error, input_value='PoissonInput_spikes.h5', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error

The results are stored in the `output` directory. The logs are also stored in the `logs` in notebooks folder.

### Spike Report Analysis using BluePySnap

We will use the example [notebook](https://github.com/openbraininstitute/snap/blob/master/doc/source/notebooks/06_spike_reports.ipynb) from BluePySnap to analyse the spike report generated by the circuit simulation.

In [ ]:
import bluepysnap

snap_simulation = bluepysnap.Simulation(simulation_config_path)
spikes = snap_simulation.spikes
print(
    spikes.time_start,
    spikes.time_stop,
    spikes.dt
)
print(spikes.population_names)

In [ ]:
spike_pop = spikes[population_name]
node_population = spike_pop.nodes

In [ ]:
filtered = spikes.filter(group={'layer':'5'}, t_start=spikes.time_start, t_stop=spikes.time_stop)
filtered.report.head()

In [ ]:
filtered.raster();

In [ ]:
filtered.firing_rate_histogram();

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

ax = plt.gca()
ax.set_xlabel("Time [ms]")
ax.set_ylabel("PSTH [Hz]")
ax.set_title(f"PSTH for group: {filtered.group}")

times = filtered.report.index

time_start = np.min(times)
time_stop = np.max(times)

# heuristic for a nice bin size (~100 spikes per bin on average)
time_binsize = min(50.0, (time_stop - time_start) / ((len(times) / 100.0) + 1.0))

bins = np.append(np.arange(time_start, time_stop, time_binsize), time_stop)
hist, bin_edges = np.histogram(times, bins=bins)
node_count = len(snap_simulation.circuit.nodes.ids(filtered.group))  # Get length of node ids for whole `group`
freq = 1.0 * hist / node_count / (0.001 * time_binsize)

# use the middle of the bins instead of the start of the bin
ax.plot(0.5 * (bin_edges[1:] + bin_edges[:-1]), freq, label="PSTH", drawstyle="steps-mid");

In [ ]:
spikes.filter().raster();

In [ ]:
spikes.filter().raster(y_axis='etype');

In [ ]:
spikes.filter().isi(binsize=20);

### Soma Report Analysis

Let's [load](https://github.com/openbraininstitute/snap/blob/master/doc/source/notebooks/07_frame_reports.ipynb) the soma report using BluePySnap and plot it.

In [ ]:
snap_simulation.reports

In [ ]:
soma_report = snap_simulation.reports['SomaVoltage']

In [ ]:
print(
    soma_report.time_start,
    soma_report.time_stop,
    soma_report.dt
)  # Gives a warning in case the dt differs from simulation.dt

In [ ]:
soma_report.population_names

In [ ]:
soma_pop = soma_report[population_name]

In [ ]:
node_population = soma_pop.nodes

In [ ]:
ids = soma_pop.node_ids
node_population.get(ids, properties=['layer','synapse_class','x','y','z']).head()

In [ ]:
filtered = soma_report.filter(group={'layer':'5'}, t_start= soma_report.time_start, t_stop= soma_report.time_stop)
filtered.report.head()